<a href="https://colab.research.google.com/github/MahanandAdimulam/Comment-Toxicity/blob/master/Comment_toxicity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [3]:
df = pd.read_csv('/content/gdrive/MyDrive/Comment toxicity data/jigsaw-toxic-comment-classification-challenge/train.csv/train.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


#1. Preprocess the data

In [4]:
from tensorflow.keras.layers import TextVectorization

In [5]:
X = df['comment_text']
y = df.iloc[:, 2:].values

In [6]:
MAX_FEATURES = 200000 # number of words in the vocab

In [7]:
def max_word(X):
  return len(X)

In [8]:
max(X.apply(max_word))

5000

In [9]:
verctorizer = TextVectorization(max_tokens=MAX_FEATURES, output_sequence_length=1800, output_mode='int')

In [10]:
verctorizer.adapt(X.values)

In [11]:
verctorizer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'to',
 'of',
 'and',
 'a',
 'you',
 'i',
 'is',
 'that',
 'in',
 'it',
 'for',
 'this',
 'not',
 'on',
 'be',
 'as',
 'have',
 'are',
 'your',
 'with',
 'if',
 'article',
 'was',
 'or',
 'but',
 'page',
 'my',
 'an',
 'from',
 'by',
 'do',
 'at',
 'about',
 'me',
 'so',
 'wikipedia',
 'can',
 'what',
 'there',
 'all',
 'has',
 'will',
 'talk',
 'please',
 'would',
 'its',
 'no',
 'one',
 'just',
 'like',
 'they',
 'he',
 'dont',
 'which',
 'any',
 'been',
 'should',
 'more',
 'we',
 'some',
 'other',
 'who',
 'see',
 'here',
 'also',
 'his',
 'think',
 'im',
 'because',
 'know',
 'how',
 'am',
 'people',
 'why',
 'edit',
 'articles',
 'only',
 'out',
 'up',
 'when',
 'were',
 'use',
 'then',
 'may',
 'time',
 'did',
 'them',
 'now',
 'being',
 'their',
 'than',
 'thanks',
 'even',
 'get',
 'make',
 'good',
 'had',
 'very',
 'information',
 'does',
 'could',
 'well',
 'want',
 'such',
 'sources',
 'way',
 'name',
 'these',
 'deletion',
 'pages',
 'first',
 'help'

In [12]:
verctorized_text = verctorizer(X.values)

In [13]:
verctorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

In [14]:
dataset = tf.data.Dataset.from_tensor_slices((verctorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps prevent bottlenecks

In [15]:
batch_x, batch_y = dataset.as_numpy_iterator().next()

In [16]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [17]:
train_generator = train.as_numpy_iterator()

#2. Create Sequential Model

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
from tensorflow.keras.callbacks import EarlyStopping

In [19]:
model = Sequential()
model.add(Embedding(MAX_FEATURES+1, 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='sigmoid'))

In [20]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirection  (None, 64)                16640     
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [ ]:
history = model.fit(train, epochs=10, validation_data=val, callbacks=EarlyStopping(monitor='loss'))

Epoch 1/10
6981/6981 [==============================] - 893s 127ms/step - loss: 0.0628 - val_loss: 0.0458
Epoch 2/10
6981/6981 [==============================] - 857s 123ms/step - loss: 0.0451 - val_loss: 0.0400
Epoch 3/10
6981/6981 [==============================] - 796s 114ms/step - loss: 0.0407 - val_loss: 0.0359
Epoch 4/10
6981/6981 [==============================] - 791s 113ms/step - loss: 0.0368 - val_loss: 0.0324
Epoch 5/10
6981/6981 [==============================] - 787s 113ms/step - loss: 0.0333 - val_loss: 0.0289
Epoch 6/10
6981/6981 [==============================] - 782s 112ms/step - loss: 0.0302 - val_loss: 0.0264
Epoch 7/10
6981/6981 [==============================] - 782s 112ms/step - loss: 0.0268 - val_loss: 0.0234
Epoch 8/10
6981/6981 [==============================] - 777s 111ms/step - loss: 0.0244 - val_loss: 0.0208
Epoch 9/10
6981/6981 [==============================] - 782s 112ms/step - loss: 0.0220 - val_loss: 0.0193
Epoch 10/10
6981/6981 [=======================

In [ ]:
model.save('comment_toxicity_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
model = tf.keras.models.load_model('comment_toxicity_model.h5')

#3. Make Predictions

In [34]:
input_text = verctorizer('You freaking suck! I am going to hit you.')

In [35]:
model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 0s 174ms/step


array([[0.9988563 , 0.5248725 , 0.99551785, 0.06247126, 0.9245769 ,
        0.10859081]], dtype=float32)

In [37]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [38]:
res = model.predict(batch_X)

1/1 [==============================] - 0s 305ms/step


#4. Evaluate the model

In [39]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [40]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [41]:
for batch in test.as_numpy_iterator():
  X_test, y_test = batch
  yhat=model.predict(X_test)
  y_test = y_test.flatten()
  yhat = yhat.flatten()

  pre.update_state(y_test, yhat)
  re.update_state(y_test, yhat)
  acc.update_state(y_test, yhat)

1/1 [==============================] - 0s 194ms/step


In [42]:
print(f'Precision : {pre.result().numpy()}, Recall : {re.result().numpy()}, Accuracy : {acc.result().numpy()}')

Precision : 0.9176837801933289, Recall : 0.9242424368858337, Accuracy : 0.4924774467945099


#5. Test and Gradio

In [44]:
pip install gradio jinja2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 9.7 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size

In [45]:
import gradio as gr

In [46]:
model = tf.keras.models.load_model('comment_toxicity_model.h5')

In [57]:
def comment_toxicity(comment):
  vectorized_comment = verctorizer([comment])
  results = model.predict(vectorized_comment)
  text = ''
  for idx, col in enumerate(df.columns[2:]):
    text += f'{col}: {results[0][idx]>0.5}\n'
  return text

In [58]:
interface = gr.Interface(fn=comment_toxicity, inputs=gr.Textbox(lines=2, placeholder='Comment to score'), outputs='text')

In [59]:
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3ec282edbf98ef33a8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
